In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from zipfile import ZipFile
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array
from keras.utils import np_utils
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense, Dropout
from keras.models import Sequential


In [ ]:
dataset = []
labels = []


In [ ]:
def prepare_benign_images(path):
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    try:
        img_res = cv2.resize(img,(256,256))
        img_array = img_to_array(img_res)
        img_array = img_array/255.0
        dataset.append(img_array)
        labels.append('0')
    except:
        print("error")

In [ ]:
def prepare_malignant_images(path):
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    try:
        img_res = cv2.resize(img,(256,256))
        img_array = img_to_array(img_res)
        img_array = img_array/255.0
        dataset.append(img_array)
        labels.append('1')
    except:
        print("error")

In [ ]:
from os import listdir
 
## pre processing train benign images
folder_dir = "/kaggle/input/breakhis-400x/BreaKHis 400X/train/benign"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        path = os.path.join('/kaggle/input/breakhis-400x/BreaKHis 400X/train/benign',images)
        prepare_benign_images(path)
        
        
folder_dir = "/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign"
for images in os.listdir(folder_dir):
    # check if the image ends with png
    if (images.endswith(".png")):
        path = os.path.join('/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign',images)
        prepare_benign_images(path)

In [ ]:
## pre processing train malignant images
folder_dir = "/kaggle/input/breakhis-400x/BreaKHis 400X/train/malignant"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        path = os.path.join('/kaggle/input/breakhis-400x/BreaKHis 400X/train/malignant',images)
        prepare_malignant_images(path)
        
folder_dir = "/kaggle/input/breakhis-400x/BreaKHis 400X/test/malignant"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        path = os.path.join('/kaggle/input/breakhis-400x/BreaKHis 400X/test/malignant',images)
        prepare_malignant_images(path)

In [ ]:
len(labels)

In [ ]:
images = np.array(dataset)
label_arr = np.array(labels)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(images,label_arr,stratify=label_arr,test_size=0.20,random_state=44)
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)


In [ ]:
import tensorflow as tf
input_shape = (128,256,256,3)
n_classes = 2

vgg19_model = tf.keras.applications.vgg19.VGG19(
    input_shape= (256,256,3),
include_top=False,
weights = 'imagenet'
)
vgg19_model.trainable = False

vgg19_model.summary()

In [ ]:
vgg19train = tf.keras.Sequential([
    vgg19_model,
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

vgg19train.summary()

In [ ]:
vgg19train.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)
history = vgg19train.fit(
    x_train,
    y_train,
    batch_size=64,
    verbose=1,
    epochs= 100 
    
)

In [ ]:
pred = vgg19train.predict(x_test)

vgg19train.evaluate(x_test,y_test)